<a href="https://colab.research.google.com/github/Andrewsas/trabalho-fundamentos-cid/blob/master/Modulo_4/Projeto_final_modulo_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports e Dependências

In [1]:
import pandas as pd
import requests

#Funções ETL

In [3]:
#Treat numeric values from dataframe
def str_to_num(df_aux, index):
  for i in index:
    col_name = df_aux.columns[i]
    df_aux[col_name] = df_aux[col_name].str.replace('.', '')
    df_aux[col_name] = df_aux[col_name].str.replace(',', '.')
    df_aux[col_name] = pd.to_numeric(df_aux[col_name])
  return df_aux


In [21]:
#Return dataframe from api informed
def retrieve_data_from_api(url, id):
  url = (url)
  response = requests.get(url)
  print("Request: {} => {}".format(id, response))
  if response.status_code == 200:
    payload = response.json()
  else:
    print(f'Error {response.status_code}: {response.reason}')
  return pd.DataFrame(payload["results"]), payload["page"], payload["total_pages"]


In [5]:
#Reference: https://www.themoviedb.org/documentation/api
def build_themoviedb_url(request_type, page):
  BASE_URL = "https://api.themoviedb.org/3/"
  API_KEY = "c9fd270fb257dd3be3a6cc9eac4b81a6"
  DEFAULT_LANGUAGE = "en-US"
  uri = BASE_URL + request_type + "?api_key=" + API_KEY + "&language=" + DEFAULT_LANGUAGE 
  if page > 0:
    uri = uri + "&page=" + str(page)
  return uri


In [17]:
def load_all_themoviedb(request):
  print(request)
  current_page = 0
  total_pages = 1
  while current_page <= total_pages:
    uri = build_themoviedb_url(request, current_page)
    df_aux, current_page, total_pages = retrieve_data_from_api(uri, current_page)
    if current_page == 1:
      print("TOTAL: ", total_pages)
      df = pd.DataFrame(df_aux)
    else:
      df = df.append(df_aux, ignore_index=True)
    current_page = current_page+1
  print("DONE!")
  return df

In [23]:
def get_netflix_dataset():
  !wget https://github.com/Andrewsas/trabalho-fundamentos-cid/blob/master/data/netflix_titles.csv?raw=true
  !ls -la
  #rename
  !mv 'netflix_titles.csv?raw=true' 'netflix_titles.csv'
  df = pd.read_csv('./netflix_titles.csv', sep = ",", usecols = ["type", "title", "director", "cast", "country", "date_added", "release_year", "rating", "duration", "listed_in", "description"])
  return df

##Métodos para checagem de **outliers**

In [8]:
def tukeys_method(df):
  # Parâmetros: series
  q1 = np.quantile(df, 0.25)
  q3 = np.quantile(df, 0.75)
  iqr = q3-q1
  inner_fence = 1.5*iqr
  outer_fence = 3*iqr

  #inner fence lower and uper end
  inner_fence_le = q1-inner_fence
  inner_fence_ue = q3+inner_fence

  #outer fence lower and uper end
  outer_fence_le = q1-outer_fence
  outer_fence_ue = q3+outer_fence

  outliers_prob = []
  outliers_poss = []
  for index, x in enumerate(df):
    if x <= outer_fence_le or x >= outer_fence_ue:
      outliers_prob.append(index)
  for index, x in enumerate(df):
    if x <= inner_fence_le or x >= inner_fence_ue:
      outliers_poss.append(index)
  return outliers_prob, outliers_poss

#Carregamento dos dados

1.   Filmes - [the movie database](https://www.themoviedb.org/)
2.   Série - [the movie database](https://www.themoviedb.org/)
3.   Netflix - [Kaggle](https://www.kaggle.com/shivamb/netflix-shows)

In [ ]:
#Movies
df_movies = load_all_themoviedb("movie/top_rated")
df_movies.info()

In [ ]:
#Series
df_series = load_all_themoviedb("tv/top_rated")
df_series.info()

In [24]:
#Netflix
df_netflix = get_netflix_dataset()
df_netflix.info()

--2021-02-21 23:14:55--  https://github.com/Andrewsas/trabalho-fundamentos-cid/blob/master/data/netflix_titles.csv?raw=true
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/Andrewsas/trabalho-fundamentos-cid/raw/master/data/netflix_titles.csv [following]
--2021-02-21 23:14:55--  https://github.com/Andrewsas/trabalho-fundamentos-cid/raw/master/data/netflix_titles.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Andrewsas/trabalho-fundamentos-cid/master/data/netflix_titles.csv [following]
--2021-02-21 23:14:56--  https://raw.githubusercontent.com/Andrewsas/trabalho-fundamentos-cid/master/data/netflix_titles.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting 